In [2]:
from utils.binaries import *
from utils.plotting import *
from utils.Auger.FD import PixelPlot, XYScanner

15:17:59 (  +66.0s) [INFO   ] -- set MONI_PATH = '/cr/work/filip/monit_and_sd/'
15:17:59 (    +1ms) [INFO   ] -- set HIST_PATH = '/cr/work/filip/monit_and_sd/'
15:17:59 (    +1ms) [INFO   ] -- set PLOT_PATH = '/cr/data01/filip/plots/'
15:17:59 (    +2ms) [INFO   ] -- set DATA_PATH = '/cr/data01/filip/Data/'
15:17:59 (    +1ms) [INFO   ] -- set HIST_PATH = '/cr/work/filip/monit_and_sd/'
15:17:59 (    +1ms) [INFO   ] -- set PLOT_PATH = '/cr/data01/filip/plots/'
15:17:59 (    +2ms) [INFO   ] -- set DATA_PATH = '/cr/data01/filip/Data/'


In [5]:
import os

SCANNER_RUNLISTS = "/cr/data01/filip/xy-calibration/config/calib_runlists/"

for LIST in os.listdir(SCANNER_RUNLISTS):
    if 'va' in LIST or 'testing' in LIST: continue

    runlist = XYScanner.load_runlist(LIST[11:18])

In [ ]:
print(runlist)